# Table Visualization

This section demonstrates visualization of tabular data using the [Styler][styler]
class. For information on visualization with charting please see [Chart Visualization][viz]. This document is written as a Jupyter Notebook, and can be viewed or downloaded [here][download].

[styler]: ../reference/api/pandas.io.formats.style.Styler.html#pandas.io.formats.style.Styler
[viz]: visualization.rst
[download]: https://nbviewer.ipython.org/github/pandas-dev/pandas/blob/master/doc/source/user_guide/style.ipynb

## Styler Object and HTML 

Styling should be performed after the data in a DataFrame has been processed. The [Styler][styler] creates an HTML `<table>` and leverages CSS styling language to manipulate many parameters including colors, fonts, borders, background, etc. See [here][w3schools] for more information on styling HTML tables. This allows a lot of flexibility out of the box, and even enables web developers to integrate DataFrames into their exiting user interface designs.
    
The `DataFrame.style` attribute is a property that returns a [Styler][styler] object. It has a `_repr_html_` method defined on it so they are rendered automatically in Jupyter Notebook.

[styler]: ../reference/api/pandas.io.formats.style.Styler.html#pandas.io.formats.style.Styler
[w3schools]: https://www.w3schools.com/html/html_tables.asp

In [126]:
import matplotlib.pyplot
# We have this here to trigger matplotlib's font cache stuff.
# This cell is hidden from the output

In [127]:
import pandas as pd
import numpy as np

df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21,19],[19, 439, 6, 452, 226,232]], 
                  index=pd.Index(['Tumor (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'), 
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

The above output looks very similar to the standard DataFrame HTML representation. But the HTML here has already attached some CSS classes to each cell, even if we haven't yet created any styles. We can view these by calling the  [.render()][render] method, which returns the raw HTML as string, which is useful for further processing or adding to a file - read on in [More about CSS and HTML](#More-About-CSS-and-HTML). Below we will show how we can use these to format the DataFrame to be more communicative. For example how we can build `s`:

[render]: ../reference/api/pandas.io.formats.style.Styler.render.html#pandas.io.formats.style.Styler.render

In [155]:
s = df.style\
      .hide_columns([('Random', 'Tumour'), ('Random', 'Non-Tumour')])\
      .format('{:.0f}')\
      .set_table_styles([
      {
        'selector': '',
        'props': 'caption-side: bottom; border-collapse: separate;'
      },{
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
      },{
        'selector': 'th:not(.index_name)',
        'props': 'background-color: darkblue; color: white;'
      },{
        'selector': 'th.col_heading',
        'props': 'text-align: center;'
      },{
        'selector': 'th.col_heading.level0',
        'props': 'font-size: 1.5em;'
      },{
        'selector': 'th.col2',
        'props': 'border-left: 1px solid white;'
      },{
        'selector': '.col2',
        'props': 'border-left: 1px solid darkblue;'
      },{
        'selector': 'td',
        'props': 'text-align: center; font-weight:bold;'
      },{
        'selector': '.true',
        'props': 'background-color: #e6ffe6;'
      },{
        'selector': '.false',
        'props': 'background-color: #ffe6e6;'
      },{
        'selector': '.border-red',
        'props': 'border: 2px dashed red;'
      },{
        'selector': '.border-green',
        'props': 'border: 2px dashed green;'
      },{
        'selector': 'td:hover',
        'props': 'background-color: #ffffb3;'
      }])\
      .set_td_classes(pd.DataFrame([['true border-green', 'false', 'true', 'false border-red', '', ''],
                                    ['false', 'true', 'false', 'true', '', '']], 
                                    index=df.index, columns=df.columns))\
      .set_caption("Confusion matrix for multiple cancer prediction models.")\
      .set_tooltips(pd.DataFrame([['DT has a very strong true positive rate', '', '', 'Regression total false negatives is unacceptable', '', ''],
                                    ['', '', '', '', '', '']], 
                                    index=df.index, columns=df.columns))\
      .set_tooltips_class(name='pd-tt', properties=
    'visibility: hidden; position: absolute; z-index: 1;'
    'background-color: lightgrey; color: darkblue; font-size: 0.8em;' 
    'transform: translate(0px, -24px); padding: 0.5em; border-radius: 0.5em;'
)


In [156]:
s

## Formatting Display Values 

Before adding styles it is useful to show that the [Styler][styler] can distinguish the *display* value from the *actual* value. To control the display value, the text is printed in each cell, and we can use [.format()][formatfunc] method to manipulate this according to a [format spec string][format] or a callable that takes a single value and returns a string. It is possible to define this for the whole table or for individual columns. 

Missing data can also be reformatted easily here and we can also [hide data](#Hiding-Data) if necessary.

[styler]: ../reference/api/pandas.io.formats.style.Styler.html#pandas.io.formats.style.Styler
[format]: https://docs.python.org/3/library/string.html#format-specification-mini-language
[formatfunc]: ../reference/api/pandas.io.formats.style.Styler.format.html#pandas.io.formats.style.Styler.format

In [157]:
df.style.format("{:,.0f}")

In [158]:
df.style.format('{:.0f}').format({
    ('Decision Tree', 'Tumour'): "{:.2f}",
    ('Regression', 'Non-Tumour'): lambda x: "{:,.1f}".format(x*-1e3),
})

### Hiding Data

The index can be hidden from rendering by calling [.hide_index()][hideidx]. 

Columns can be hidden from rendering by calling [.hide_columns()][hidecols] and passing in the name of a column, or a slice of columns.

Hiding does not change the integer arrangement of CSS classes, i.e. the `Employees` column is not be reindexed to `.col0` but remains as `.col2`. It does also not affect the underlying ``Styler.data``.

We can update our `style` object to hide some data and format the values...

[hideidx]: ../reference/api/pandas.io.formats.style.Styler.hide_index.html#pandas.io.formats.style.Styler.hide_index
[hidecols]: ../reference/api/pandas.io.formats.style.Styler.hide_columns.html#pandas.io.formats.style.Styler.hide_columns

In [161]:
s = df.style.format('{:.0f}').hide_columns([('Random', 'Tumour'), ('Random', 'Non-Tumour')])
s

## Methods to Add Styles

There are **3 primary methods of adding custom CSS styles to [Styler][styler]**:

- **Using [.set_table_styles()][table]** to control broader areas of the table with specified internal CSS. Although table styles allow the flexibility to add CSS selectors and properties controlling all individual parts of the table, they are unwieldy for individual cell specifications. Also, note that table styles cannot be exported to Excel.  
- **Using [.set_td_classes()][td_class]** to directly link either external CSS classes to your data cells or link the internal CSS classes created by [.set_table_styles()][table]. These cannot be used on column header rows or indexes, and also won't export to Excel. 
- **Using the [.apply()][apply] and [.applymap()][applymap] functions** to add direct internal CSS to specific data cells. These cannot be used on column header rows or indexes, but only these methods add styles that will export to Excel. These methods work in a similar way to [DataFrame.apply()][dfapply] and [DataFrame.applymap()][dfapplymap].

Next we cover the advantages and limitations of each of the above methods.

[table]: ../reference/api/pandas.io.formats.style.Styler.set_table_styles.html#pandas.io.formats.style.Styler.set_table_styles
[styler]: ../reference/api/pandas.io.formats.style.Styler.html#pandas.io.formats.style.Styler
[td_class]: ../reference/api/pandas.io.formats.style.Styler.set_td_classes.html#pandas.io.formats.style.Styler.set_td_classes
[apply]: ../reference/api/pandas.io.formats.style.Styler.apply.html#pandas.io.formats.style.Styler.apply
[applymap]: ../reference/api/pandas.io.formats.style.Styler.applymap.html#pandas.io.formats.style.Styler.applymap
[dfapply]: ../reference/api/pandas.DataFrame.apply.html#pandas.DataFrame.apply
[dfapplymap]: ../reference/api/pandas.DataFrame.applymap.html#pandas.DataFrame.applymap

## Using Table Styles

Table styles are flexible enough to control all individual parts of the table, including column headers and indexes. 
However, they can be unwieldy to type for individual data cells or for any kind of conditional formatting, so we recommend that table styles are used for broad styling, such as rows or columns at a time.

Table styles are also used to control features which can apply to the whole table at once such as greating a generic hover functionality. This `:hover` pseudo-selectors, as well as others, can only be used this way.

CSS style sheets are typically composed of selectors, e.g. element ids or css classes, and attribute value pairs:

```
tr:hover {
  background-color: #ffff99;
}
```

The necessary format to pass styles to [.set_table_styles()][table] is as a list of dicts, each with a CSS-selector tag and CSS-properties. Properties can either be a list of 2-tuples, or a regular CSS-string For example:

[table]: ../reference/api/pandas.io.formats.style.Styler.set_table_styles.html#pandas.io.formats.style.Styler.set_table_styles

In [ ]:
hover_style = {
    'selector': "tr:hover", 
    'props': [("background-color", "#ffff99")]
}
header_style = {
    'selector': "th", 
    'props': 'font-size: 115%; text-align: center; color: #e83e8c;'
}
df.style.format("{:,.0f}").set_table_styles([hover_style, header_style])

Table styles allows for controlling specific rows and columns by their integer index, for example

In [ ]:
df.style.format("{:,.0f}").set_table_styles([hover_style, header_style, {
    'selector': 'th.level1',
    'props': 'color: darkblue;'
}])

As a convenience method (*since version 1.2.0*) we can also pass a **dict** to [.set_table_styles()][table] which contains row or column keys. Behind the scenes Styler just indexes the keys and add relevant `.col` or `.row` classes as necessary to the CSS selectors.

Table styles can be chained provided multiple methods don't *overwrite*.

[table]: ../reference/api/pandas.io.formats.style.Styler.set_table_styles.html#pandas.io.formats.style.Styler.set_table_styles

In [ ]:
s = df.style.format("{:,.0f}")\
  .set_table_styles([hover_style, header_style, {
    'selector': 'th.level1',
    'props': 'color: darkblue;'}])\
  .set_table_styles({
    'Employees': [{
        'selector': 'td',
        'props': 'font-weight: bold;'}]
    }, axis=0, overwrite=False)\
  .set_table_styles({
    (20, 'Mar'): [{
        'selector': '',
        'props': 'background-color: darkgrey; color: white;'}]
    }, axis=1, overwrite=False)
s

## Setting Classes and Linking to External CSS

*New in version 1.2.0*

If you have designed a website then it is likely you will already have an external CSS file that controls the styling of table and cell objects within your website. You may want to use these native files rather than duplicate all the CSS in python.

For example, suppose we have an external CSS which controls table properties and has some additional classes to style individual elements (*here we manually add one to the HTML of this notebook page for demonstration*):

In [ ]:
# from IPython.display import HTML
# style = \
# "<style>"\
# ".table-cls {width: 100%;}"\
# ".cls-invert {background-color: darkgrey; color: white;}"\
# ".cls-bold {font-weight: bold; font-size: 120%;}"\
# "</style>"
# HTML(style)

<style>
    .table-cls {width: 100%; border: 1px solid red;}
    .nbinput .prompt,
    .nboutput .prompt {
        display: none;
    }
</style>    

At a minimum we can add a class attribute to our `<table>` element, which will connect with the external CSS, and in this case change the width of the table.

In [ ]:
s.set_table_attributes('class="table-cls"')

### Data Cell CSS Classes

The [.set_td_classes()][tdclass] method accepts a DataFrame with matching indices and columns to the underlying [Styler][styler]'s DataFrame. That DataFrame will contain strings as css-classes to add to individual data cells: the `<td>` elements of the `<table>`.

[tdclass]: ../reference/api/pandas.io.formats.style.Styler.set_td_classes.html#pandas.io.formats.style.Styler.set_td_classes
[styler]: ../reference/api/pandas.io.formats.style.Styler.html#pandas.io.formats.style.Styler

In [ ]:
df = pd.DataFrame(np.random.randn(5,5), columns=[f'Trial {x}' for x in range(5)])
df.iloc[3,4], df.iloc[4,3] = np.nan, np.nan
df.style.format('{:,.3f}')

In [ ]:
cls_inv = pd.DataFrame('', index=df.index, columns=df.columns)
cls_bold = pd.DataFrame('', index=df.index, columns=df.columns)
cls_inv.iloc[0:2, 0:2] = 'cls-invert '
cls_bold.iloc[1:3, 1:3] = 'cls-bold '
df.style.format('{:,.3f}').set_td_classes(cls_inv + cls_bold)

It is possible for this to be an entirely internal CSS solution by defining the class within table styles.

In [ ]:
df.style.format('{:,.3f}')\
  .set_table_styles([
    {'selector': '.cls-invert',
     'props': [('color', 'white'), ('background-color', '#e83e8c')]}])\
  .set_td_classes(cls_inv)

The **advantage** of linking to external CSS is that it can be applied very easily. One can build a DataFrame of (multiple) CSS classes to add to each cell dynamically using traditional `DataFrame.apply` and `DataFrame.applymap` methods, or otherwise, and then add those to the Styler. It will integrate with your website's existing CSS styling.

The **disadvantage** of this approach is that it is not easy to transmit files standalone. For example the external CSS must be included or the styling will simply be lost. It is also, as this example shows, not well suited (at a table level) for Jupyter Notebooks. Also this method cannot be used for exporting to Excel, for example, since the external CSS cannot be referenced either by the exporters or by Excel itself.

In [ ]:
df.style.set_table_styles({
    'A': [{'selector': '',
           'props': [('color', 'red')]}],
    'B': [{'selector': 'td',
           'props': [('color', 'blue')]}]
}, axis=0)

In [ ]:
df.style.set_table_styles({
    3: [{'selector': 'td',
           'props': [('color', 'green')]}]
}, axis=1)

We can also chain all of the above by setting the `overwrite` argument to `False` so that it preserves previous settings.

In [ ]:
from pandas.io.formats.style import Styler
s = Styler(df, cell_ids=False, uuid_len=0).\
    set_table_styles(styles).\
    set_table_styles({
        'A': [{'selector': '',
               'props': [('color', 'red')]}],
        'B': [{'selector': 'td',
               'props': [('color', 'blue')]}]
    }, axis=0, overwrite=False).\
    set_table_styles({
        3: [{'selector': 'td',
             'props': [('color', 'green')]}]
    }, axis=1, overwrite=False)
s

By using these `table_styles` and the additional `Styler` arguments to optimize the HTML we have compressed these styles to only a few lines withing the \<style\> tags and none of the \<td\> cells require any `id` attributes.  

In [ ]:
s.render().split('\n')[:16]

The **advantage** of table styles is obviously the reduced HTML that it can create and the relative ease with which more general parts of the table can be quickly styled, e.g. by applying a generic hover, rather than having to apply a hover to each cell individually. Rows and columns as individual objects can only be styled in this way.

The **disadvantage** of being restricted solely to table styles is that you have very limited ability to target and style individual cells based on dynamic criteria. For this, one must use either of the other two methods. Also table level styles cannot be exported to Excel: to format cells for Excel output you must use the Styler Functions method below.

### Styler Functions

Thirdly we can use the method to pass your style functions into one of the following methods:

- ``Styler.applymap``: elementwise
- ``Styler.apply``: column-/row-/table-wise

Both of those methods take a function (and some other keyword arguments) and applies your function to the DataFrame in a certain way.
`Styler.applymap` works through the DataFrame elementwise.
`Styler.apply` passes each column or row into your DataFrame one-at-a-time or the entire table at once, depending on the `axis` keyword argument.
For columnwise use `axis=0`, rowwise use `axis=1`, and for the entire table at once use `axis=None`.

For `Styler.applymap` your function should take a scalar and return a single string with the CSS attribute-value pair.

For `Styler.apply` your function should take a Series or DataFrame (depending on the axis parameter), and return a Series or DataFrame with an identical shape where each value is a string with a CSS attribute-value pair.

The **advantage** of this method is that there is full granular control and the output is isolated and easily transferrable, especially in Jupyter Notebooks.

The **disadvantage** is that the HTML/CSS required to produce this needs to be directly generated from the Python code and it can lead to inefficient data transfer for large tables.

Let's see some examples.

Let's write a simple style function that will color negative numbers red and positive numbers black.

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In this case, the cell's style depends only on its own value.
That means we should use the `Styler.applymap` method which works elementwise.

In [ ]:
s = df.style.applymap(color_negative_red)
s

Notice the similarity with the standard `df.applymap`, which operates on DataFrames elementwise. We want you to be able to reuse your existing knowledge of how to interact with DataFrames.

Notice also that our function returned a string containing the CSS attribute and value, separated by a colon just like in a `<style>` tag. This will be a common theme.

Finally, the input shapes matched. `Styler.applymap` calls the function on each scalar input, and the function returns a scalar output.

Now suppose you wanted to highlight the maximum value in each column.
We can't use `.applymap` anymore since that operated elementwise.
Instead, we'll turn to `.apply` which operates columnwise (or rowwise using the `axis` keyword). Later on we'll see that something like `highlight_max` is already defined on `Styler` so you wouldn't need to write this yourself.

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [ ]:
df.style.apply(highlight_max)

In this case the input is a `Series`, one column at a time.
Notice that the output shape of `highlight_max` matches the input shape, an array with `len(s)` items.

A common use case is also to highlight values based on comparison between columns. Suppose we wish to highlight those cells in columns 'B' and 'C' which are lower than respective values in 'E' then we can write a comparator function. (You can read a little more below in 'Finer Control: Slicing')

In [ ]:
def compare_col(s, comparator=None):
    attr = 'background-color: #00BFFF;'
    return np.where(s < comparator, attr, '')

In [ ]:
df.style.apply(compare_col, subset=['B', 'C'], comparator=df['E'])

We encourage you to use method chains to build up a style piecewise, before finally rending at the end of the chain. Note the ordering of application will affect styles that overlap.

In [ ]:
df.style.\
    apply(compare_col, subset=['B', 'C'], comparator=df['E']).\
    applymap(color_negative_red).\
    apply(highlight_max)

Above we used `Styler.apply` to pass in each column one at a time.

<span style="background-color: #DEDEBE">*Debugging Tip*: If you're having trouble writing your style function, try just passing it into <code style="background-color: #DEDEBE">DataFrame.apply</code>. Internally, <code style="background-color: #DEDEBE">Styler.apply</code> uses <code style="background-color: #DEDEBE">DataFrame.apply</code> so the result should be the same.</span>

What if you wanted to highlight just the maximum value in the entire table?
Use `.apply(function, axis=None)` to indicate that your function wants the entire table, not one column or row at a time. Let's try that next.

We'll rewrite our `highlight-max` to handle either Series (from `.apply(axis=0 or 1)`) or DataFrames (from `.apply(axis=None)`). We'll also allow the color to be adjustable, to demonstrate that `.apply`, and `.applymap` pass along keyword arguments.

In [ ]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

When using ``Styler.apply(func, axis=None)``, the function must return a DataFrame with the same index and column labels.

In [ ]:
s = df.style.apply(highlight_max, color='darkorange', axis=None)
s

### Building Styles Summary

Style functions should return strings with one or more CSS `attribute: value` delimited by semicolons. Use

- `Styler.applymap(func)` for elementwise styles
- `Styler.apply(func, axis=0)` for columnwise styles
- `Styler.apply(func, axis=1)` for rowwise styles
- `Styler.apply(func, axis=None)` for tablewise styles

And crucially the input and output shapes of `func` must match. If `x` is the input then ``func(x).shape == x.shape``.

## Optimization

The HTML ....

## Tooltips

*New in version 1.3.0*

You can now add tooltips in the same way you can add external CSS classes to datacells by providing a string based DataFrame with intersecting indices and columns.

In [ ]:
tt = pd.DataFrame(data=[[None, 'No Data', None], 
                     [None, None, 'Missing Data'], 
                     ['Maximum value across entire DataFrame', None, None]], 
                  index=[0, 3, 9], 
                  columns=['A', 'C', 'D'])
s.set_tooltips(tt)

The tooltips are added with a default CSS styling, however, you have full control of the tooltips in the following way. The name of the class can be integrated with your existing website's CSS so you do not need to set any properties within Python if you have the external CSS files. 

In [ ]:
s.set_tooltips_class(name='pd-tt', properties=[
    ('visibility', 'hidden'),
    ('position', 'absolute'),
    ('z-index', '1'),
    ('background-color', 'blue'),
    ('color', 'white'),
    ('font-size', '1.5em'),
    ('transform', 'translate(3px, -11px)'),
    ('padding', '0.5em'),
    ('border', '1px solid red'),
    ('border-radius', '0.5em')
])

## Finer control: slicing

Both `Styler.apply`, and `Styler.applymap` accept a `subset` keyword.
This allows you to apply styles to specific rows or columns, without having to code that logic into your `style` function.

The value passed to `subset` behaves similar to slicing a DataFrame.

- A scalar is treated as a column label
- A list (or series or numpy array)
- A tuple is treated as `(row_indexer, column_indexer)`

Consider using `pd.IndexSlice` to construct the tuple for the last one.

In [ ]:
df.style.apply(highlight_max, subset=['B', 'C', 'D'])

For row and column slicing, any valid indexer to `.loc` will work.

In [ ]:
df.style.applymap(color_negative_red,
                  subset=pd.IndexSlice[2:5, ['B', 'D']])

Only label-based slicing is supported right now, not positional.

If your style function uses a `subset` or `axis` keyword argument, consider wrapping your function in a `functools.partial`, partialing out that keyword.

```python
my_func2 = functools.partial(my_func, subset=42)
```

## Finer Control: Display Values

We distinguish the *display* value from the *actual* value in `Styler`.
To control the display value, the text is printed in each cell, use `Styler.format`. Cells can be formatted according to a [format spec string](https://docs.python.org/3/library/string.html#format-specification-mini-language) or a callable that takes a single value and returns a string.

In [ ]:
df.style.format("{:.2%}")

Use a dictionary to format specific columns.

In [ ]:
df.style.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})

Or pass in a callable (or dictionary of callables) for more flexible handling.

In [ ]:
df.style.format({"B": lambda x: "±{:.2f}".format(abs(x))})

You can format the text displayed for missing values by `na_rep`.

In [ ]:
df.style.format("{:.2%}", na_rep="-")

These formatting techniques can be used in combination with styling.

In [ ]:
df.style.highlight_max().format(None, na_rep="-")

## Builtin styles

Finally, we expect certain styling functions to be common enough that we've included a few "built-in" to the `Styler`, so you don't have to write them yourself.

In [ ]:
df.style.highlight_null(null_color='red')

You can create "heatmaps" with the `background_gradient` method. These require matplotlib, and we'll use [Seaborn](https://stanford.edu/~mwaskom/software/seaborn/) to get a nice colormap.

In [ ]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

s = df.style.background_gradient(cmap=cm)
s

`Styler.background_gradient` takes the keyword arguments `low` and `high`. Roughly speaking these extend the range of your data by `low` and `high` percent so that when we convert the colors, the colormap's entire range isn't used. This is useful so that you can actually read the text still.

In [ ]:
# Uses the full color range
df.loc[:4].style.background_gradient(cmap='viridis')

In [ ]:
# Compress the color range
(df.loc[:4]
    .style
    .background_gradient(cmap='viridis', low=.5, high=0)
    .highlight_null('red'))

There's also `.highlight_min` and `.highlight_max`.

In [ ]:
df.style.highlight_max(axis=0)

Use `Styler.set_properties` when the style doesn't actually depend on the values.

In [ ]:
df.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

### Bar charts

You can include "bar charts" in your DataFrame.

In [ ]:
df.style.bar(subset=['A', 'B'], color='#d65f5f')

New in version 0.20.0 is the ability to customize further the bar chart: You can now have the `df.style.bar` be centered on zero or midpoint value (in addition to the already existing way of having the min value at the left side of the cell), and you can pass a list of `[color_negative, color_positive]`.

Here's how you can change the above with the new `align='mid'` option:

In [ ]:
df.style.bar(subset=['A', 'B'], align='mid', color=['#d65f5f', '#5fba7d'])

The following example aims to give a highlight of the behavior of the new align options:

In [ ]:
import pandas as pd
from IPython.display import HTML

# Test series
test1 = pd.Series([-100,-60,-30,-20], name='All Negative')
test2 = pd.Series([10,20,50,100], name='All Positive')
test3 = pd.Series([-10,-5,0,90], name='Both Pos and Neg')

head = """
<table>
    <thead>
        <th>Align</th>
        <th>All Negative</th>
        <th>All Positive</th>
        <th>Both Neg and Pos</th>
    </thead>
    </tbody>

"""

aligns = ['left','zero','mid']
for align in aligns:
    row = "<tr><th>{}</th>".format(align)
    for series in [test1,test2,test3]:
        s = series.copy()
        s.name=''
        row += "<td>{}</td>".format(s.to_frame().style.bar(align=align, 
                                                           color=['#d65f5f', '#5fba7d'], 
                                                           width=100).render()) #testn['width']
    row += '</tr>'
    head += row
    
head+= """
</tbody>
</table>"""
        

HTML(head)

## Sharing styles

Say you have a lovely style built up for a DataFrame, and now you want to apply the same style to a second DataFrame. Export the style with `df1.style.export`, and import it on the second DataFrame with `df1.style.set`

In [ ]:
df2 = -df
style1 = df.style.applymap(color_negative_red)
style1

In [ ]:
style2 = df2.style
style2.use(style1.export())
style2

Notice that you're able to share the styles even though they're data aware. The styles are re-evaluated on the new DataFrame they've been `use`d upon.

## Other Options

You've seen a few methods for data-driven styling.
`Styler` also provides a few other options for styles that don't depend on the data.

- precision
- captions
- table-wide styles
- missing values representation
- hiding the index or columns

Each of these can be specified in two ways:

- A keyword argument to `Styler.__init__`
- A call to one of the `.set_` or `.hide_` methods, e.g. `.set_caption` or `.hide_columns`

The best method to use depends on the context. Use the `Styler` constructor when building many styled DataFrames that should all share the same properties. For interactive use, the`.set_` and `.hide_` methods are more convenient.

### Precision

You can control the precision of floats using pandas' regular `display.precision` option.

In [ ]:
with pd.option_context('display.precision', 2):
    html = (df.style
              .applymap(color_negative_red)
              .apply(highlight_max))
html

Or through a `set_precision` method.

In [ ]:
df.style\
  .applymap(color_negative_red)\
  .apply(highlight_max)\
  .set_precision(2)

Setting the precision only affects the printed number; the full-precision values are always passed to your style functions. You can always use `df.round(2).style` if you'd prefer to round from the start.

### Captions

Regular table captions can be added and, if necessary, controlled with CSS.

In [ ]:
df.style.set_caption('Colormaps, with a caption.')\
    .set_table_styles([{
        'selector': "caption", 'props': [("caption-side", "bottom")]
    }])\
    .background_gradient(cmap=cm)

### Missing values

You can control the default missing values representation for the entire table through `set_na_rep` method.

In [ ]:
(df.style
   .set_na_rep("FAIL")
   .format(None, na_rep="PASS", subset=["D"])
   .highlight_null("yellow"))

### Hiding the Index or Columns

The index can be hidden from rendering by calling `Styler.hide_index`. Columns can be hidden from rendering by calling `Styler.hide_columns` and passing in the name of a column, or a slice of columns.

In [ ]:
df.style.hide_index()

In [ ]:
df.style.hide_columns(['C','D'])

### Limitations

- DataFrame only `(use Series.to_frame().style)`
- The index and columns must be unique
- No large repr, and performance isn't great; this is intended for summary DataFrames
- You can only style the *values*, not the index or columns (except with `table_styles` above)
- You can only apply styles, you can't insert new HTML entities

Some of these will be addressed in the future.
Performance can suffer when adding styles to each cell in a large DataFrame.
It is recommended to apply table or column based styles where possible to limit overall HTML length, as well as setting a shorter UUID to avoid unnecessary repeated data transmission. 


### Terms

- Style function: a function that's passed into `Styler.apply` or `Styler.applymap` and returns values like `'css attribute: value'`
- Builtin style functions: style functions that are methods on `Styler`
- table style: a dictionary with the two keys `selector` and `props`. `selector` is the CSS selector that `props` will apply to. `props` is a list of `(attribute, value)` tuples. A list of table styles passed into `Styler`.

## Fun stuff

Here are a few interesting examples.

`Styler` interacts pretty well with widgets. If you're viewing this online instead of running the notebook yourself, you're missing out on interactively adjusting the color palette.

In [ ]:
from IPython.html import widgets
@widgets.interact
def f(h_neg=(0, 359, 1), h_pos=(0, 359), s=(0., 99.9), l=(0., 99.9)):
    return df.style.background_gradient(
        cmap=sns.palettes.diverging_palette(h_neg=h_neg, h_pos=h_pos, s=s, l=l,
                                            as_cmap=True)
    )

In [ ]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [ ]:
np.random.seed(25)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
bigdf = pd.DataFrame(np.random.randn(20, 25)).cumsum()

bigdf.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(2)\
    .set_table_styles(magnify())

## Export to Excel

*New in version 0.20.0*

<span style="color: red">*Experimental: This is a new feature and still under development. We'll be adding features and possibly making breaking changes in future releases. We'd love to hear your feedback.*</span>

Some support is available for exporting styled `DataFrames` to Excel worksheets using the `OpenPyXL` or `XlsxWriter` engines. CSS2.2 properties handled include:

- `background-color`
- `border-style`, `border-width`, `border-color` and their {`top`, `right`, `bottom`, `left` variants}
- `color`
- `font-family`
- `font-style`
- `font-weight`
- `text-align`
- `text-decoration`
- `vertical-align`
- `white-space: nowrap`


- Only CSS2 named colors and hex colors of the form `#rgb` or `#rrggbb` are currently supported.
- The following pseudo CSS properties are also available to set excel specific style properties:
    - `number-format`

Table level styles are not included in the export to Excel: individual cells must have their properties mapped by the `Styler.apply` and/or `Styler.applymap` methods.

In [ ]:
df.style.\
    applymap(color_negative_red).\
    apply(highlight_max).\
    to_excel('styled.xlsx', engine='openpyxl')

A screenshot of the output:

![Excel spreadsheet with styled DataFrame](../_static/style-excel.png)


## Extensibility

The core of pandas is, and will remain, its "high-performance, easy-to-use data structures".
With that in mind, we hope that `DataFrame.style` accomplishes two goals

- Provide an API that is pleasing to use interactively and is "good enough" for many tasks
- Provide the foundations for dedicated libraries to build on

If you build a great library on top of this, let us know and we'll [link](https://pandas.pydata.org/pandas-docs/stable/ecosystem.html) to it.

### Subclassing

If the default template doesn't quite suit your needs, you can subclass Styler and extend or override the template.
We'll show an example of extending the default template to insert a custom header before each table.

In [ ]:
from jinja2 import Environment, ChoiceLoader, FileSystemLoader
from IPython.display import HTML
from pandas.io.formats.style import Styler

We'll use the following template:

In [ ]:
with open("templates/myhtml.tpl") as f:
    print(f.read())

Now that we've created a template, we need to set up a subclass of ``Styler`` that
knows about it.

In [ ]:
class MyStyler(Styler):
    env = Environment(
        loader=ChoiceLoader([
            FileSystemLoader("templates"),  # contains ours
            Styler.loader,  # the default
        ])
    )
    template = env.get_template("myhtml.tpl")

Notice that we include the original loader in our environment's loader.
That's because we extend the original template, so the Jinja environment needs
to be able to find it.

Now we can use that custom styler. It's `__init__` takes a DataFrame.

In [ ]:
MyStyler(df)

Our custom template accepts a `table_title` keyword. We can provide the value in the `.render` method.

In [ ]:
HTML(MyStyler(df).render(table_title="Extending Example"))

For convenience, we provide the `Styler.from_custom_template` method that does the same as the custom subclass.

In [ ]:
EasyStyler = Styler.from_custom_template("templates", "myhtml.tpl")
EasyStyler(df)

Here's the template structure:

In [ ]:
with open("templates/template_structure.html") as f:
    structure = f.read()
    
HTML(structure)

See the template in the [GitHub repo](https://github.com/pandas-dev/pandas) for more details.

In [ ]:
# Hack to get the same style in the notebook as the
# main site. This is hidden in the docs.
from IPython.display import HTML
with open("themes/nature_with_gtoc/static/nature.css_t") as f:
    css = f.read()
    
HTML('<style>{}</style>'.format(css))

## More About CSS and HTML

Cascading Style Sheet (CSS) language, which is designed to influence how a browser renders HTML elements, has its own peculiarities. It never reports errors: it just silently ignores them and doesn't render your objects how you intend so can sometimes be frustrating. Here is a very brief primer on how ``Styler`` creates HTML and interacts with CSS, with advice on common pitfalls to avoid.

## CSS Classes and Ids

The precise structure of the CSS `class` attached to each cells is as follows.

- Cells with Index and Column names include `index_name` and `level<k>` where `k` is its level in a MultiIndex
- Index label cells include
  + `row_heading`
  + `level<k>` where `k` is the level in a MultiIndex
  + `row<m>` where `m` is the numeric position of the row
- Column label cells include
  + `col_heading`
  + `level<k>` where `k` is the level in a MultiIndex
  + `col<n>` where `n` is the numeric position of the column
- Data cells include
  + `data`
  + `row<m>`, where `m` is the numeric position of the cell.
  + `col<n>`, where `n` is the numeric position of the cell.
- Blank cells include `blank`

The structure of the `id` is `T_uuid_level<k>_row<m>_col<n>` where `level<k>` is used only on headings, and headings will only have either `row<m>` or `col<n>` whichever is needed. By default we've also prepended each row/column identifier with a UUID unique to each DataFrame so that the style from one doesn't collide with the styling from another within the same notebook or page. You can read more about the use of UUIDs in [Optimization](#Optimization).

We can see example of the HTML by calling the [.render()][render] method.

[render]: ../reference/api/pandas.io.formats.style.Styler.render.html#pandas.io.formats.style.Styler.render

In [3]:
print(pd.DataFrame([[1,2],[3,4]], index=['i1', 'i2'], columns=['c1', 'c2']).style.render())

<style type="text/css">
</style>
<table id="T_1003a_">
  <thead>
    <tr>
      <th class="blank level0" ></th>
      <th class="col_heading level0 col0" >c1</th>
      <th class="col_heading level0 col1" >c2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_1003a_level0_row0" class="row_heading level0 row0" >i1</th>
      <td id="T_1003a_row0_col0" class="data row0 col0" >1</td>
      <td id="T_1003a_row0_col1" class="data row0 col1" >2</td>
    </tr>
    <tr>
      <th id="T_1003a_level0_row1" class="row_heading level0 row1" >i2</th>
      <td id="T_1003a_row1_col0" class="data row1 col0" >3</td>
      <td id="T_1003a_row1_col1" class="data row1 col1" >4</td>
    </tr>
  </tbody>
</table>

